# Exercise 9: series

* pandas series vs numpy arrays [explanation](https://jakevdp.github.io/PythonDataScienceHandbook/03.01-introducing-pandas-objects.html)

### Common series operations
These are the most common series operations we use. Refer to the `pandas` docs for even more!

* Getting dates, hours, minutes from datetime types (`df.datetime_col.dt.date`)
* Parsing strings (`df.string_col.str.split('_')`)

### Common geoseries operations
These are the most common. Refer to the `geopandas` docs for even more!

* `distance` between 2 points or a point to a polygon or line [docs](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.distance.html)
* `intersects`: [docs](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.intersects.html)
* `within`: [docs](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.within.html)
* `contains`: [docs](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.contains.html)

In fact, we've often used geoseries methods without even realizing it. Often, we'd create a new column that stores either the line's length or a polygon's area. `gdf.geometry` is a geoseries, and we call methods on that geoseries, and add that as a new column.

For calculations like `length`, `area`, and `distance`, we need to use a projected CRS that has units like meters or feet. We cannot use decimal degrees (do not use WGS 84 / EPSG:3326)! Distance calculations must be done only once the spherical 3D Earth has been converted into a 2D plane.

* `length`: get the length of a line (`gdf.geometry.length`)
* `area`: get the area of a polygon (`gdf.geometry.area`)
* `centroid`: get the centroid of a polygon (`gdf.geometry.centroid`)
* `x`: get the x coordinate of a point (`gdf.geometry.x`)
* `y`: get the y coordinate of a point (`gdf.geometry.y`)

### Arrays
* Occasionally, we may even use arrays, especially when the datasets get even larger but we have simple mathematical calculations
* If we need to apply an exponential decay function to a distance column, we essentially want to multiple `distance` by some number
* Since this exponential decay function is somewhat custom and requires us to write our own formula, we would extract the column as a series (`df.distance`) and multiply each value by some other number.
* Even quicker is to use `numpy` with `distance_array = np.array(df.distance)` and get `exponential_array = distance_array*some_number`

In [1]:
import geopandas as gpd
import intake
import numpy as np
import pandas as pd

catalog = intake.open_catalog(
    "../_shared_utils/shared_utils/shared_data_catalog.yml")

If you're asking how far is a transit stop from the interstate, you'd want the distance of every point (every row) compared to an interstate highway geometry.

Let's prep the datasets to use series / geoseries to do this.

In [ ]:
stops = catalog.ca_transit_stops.read()[["agency", "stop_id", 
                                         "stop_name", "geometry"]]
highways = catalog.state_highway_network.read()

Since we want to know the distance from a stop's point to the interstate generally, we need a dissolve. We don't want to compare the distance against the I-5, the I-10 individually, but to the interstate system as a whole.

In [ ]:
interstates = (highways[highways.RouteType=="Interstate"]
               .dissolve()
               .reset_index()
               [["geometry"]]
              )

In [ ]:
df_check(stops)

In [ ]:
df_check(highways)

In [ ]:
highways.plot()

In [ ]:
# This is still a gdf, just with 1 column
type(interstates)

In [ ]:
df_check(interstates)

In [ ]:
# Pulling out the individual column, it becomes a series/geoseries.
# It's a geoseries here because we had a gdf. 
# If it was a df, it would be a series.
print(type(stops.geometry))
print(type(interstates.geometry))

Distance is something you can calculate using `geopandas`.

Specifically, it takes a geoseries on the left, and either a geoseries or a single geometry on the right.

An example of having 2 geoseries would be comparing the distance between 2 points. On the left, it would be a geoseries of the origin points and on the right, destination points.

In [ ]:
# We get a warning if we leave it in EPSG:4326!
stops.geometry.distance(interstates.geometry.iloc[0])

In [ ]:
stops_geom = stops.to_crs("EPSG:2229").geometry
interstates_geom = interstates.to_crs("EPSG:2229").geometry.iloc[0]

In [ ]:
df_check(stops_geom)

In [ ]:
interstates_geom

In [ ]:
distance_series = stops_geom.distance(interstates_geom)

In [ ]:
#returns a series (like a 1 col table)
df_check(distance_series)


In [ ]:
# Let's make sure that for every stop, a distance is calculated
print(f"# rows in stops: {len(stops_geom)}")
print(f"# rows in stops: {len(distance_series)}")

In [ ]:
# distance is numeric, not a geometry, so we're back to being a series
type(distance_series)

What can we do with this? 

We usually add it as a new column. Since we did nothing to shift the index, we can just attach the series back to our gdf.

Getting a distance calculation using geoseries is much quicker than a row-wise lambda function where you calculate the distance.

```
Alternative method that's slower:
      
interstate_geom = interstates.geometry.iloc[0]

stops = stops.assign(
   distance = stops.geometry.apply(
         lambda x: x.distance(interstate_geom))
)   
```

In [ ]:
#adds a new column to stops called `distance_to_interstate` and fills it with values from `distance_series`. 
#the indicies are the same for both, meaning they match up

stops = stops.assign(
    distance_to_interstate = distance_series
)

In [ ]:
df_check(stops)

In [ ]:
#this cell took a loooooooong time to run
#%%timeit
#distance_series = stops_geom.distance(interstates_geom)

In [ ]:
#also took a loooooong time to run
#%%timeit
#stops.assign(
   #distance = stops.geometry.apply(
       #  lambda x: x.distance(interstates_geom))
#)   

In [ ]:
#import dask_geopandas as dg

#stops_gddf = dg.from_geopandas(stops, npartitions=2)
#stops_geom_dg = stops_gddf.to_crs("EPSG:2229").geometry

In [ ]:
#was a lot laster to run
#%%timeit

#distance_series = stops_geom_dg.distance(interstates_geom)

## To Do

* Use the `stop_times` table and `stops` table.
* Calculate the straight line distance between the first and last stop for each trip. Call this column `trip_distance`
* Calculate the distance between each stop to the nearest interstate. For each trip, keep the value for the stop that's the closest to the interstate. Call this column `shortest_distance_hwy`.
* For each trip, add these 2 new columns, but use series, geoseries, and/or arrays to assign it.
* Provide a preview of the resulting df (do not export)

In [2]:
GCS_FILE_PATH = ("gs://calitp-analytics-data/data-analyses/"
                 "rt_delay/compiled_cached_views/"
                )

analysis_date = "2023-01-18"
STOP_TIMES_FILE = f"{GCS_FILE_PATH}st_{analysis_date}.parquet"
STOPS_FILE = f"{GCS_FILE_PATH}stops_{analysis_date}.parquet"
highways = catalog.state_highway_network.read()

In [3]:
#import parquet files
stops = pd.read_parquet(STOPS_FILE)
stop_times = pd.read_parquet(STOP_TIMES_FILE)

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
#grabbed ca map from previous exercise. so i can clip everything to CA if needed
districts = catalog.caltrans_districts.read().to_crs('EPSG:2229')


In [5]:
#function that runs the same checks for dfs
def df_check(x):
    display(f'shape of df:{x.shape}'),
    display(f'type of :{type(x)}'),
    display(x.head()),
    return 

In [ ]:
#what does each row mean?
#each row is a stop_key, a stop_key can have multiple feeds and stops
#what is the difference between stop_key and stop_id?

#noticed the geometry col is in WKB. need to convert this to something else.

df_check(stops)

In [7]:
#found method to create geoseries from wkb.
test = gpd.GeoSeries.from_wkb(stops.geometry)


In [8]:
#have a geoeries called `test`. now i am able to add this series back to initial stops table (using assign)
stops2 = stops.assign(pt_geom = test)

In [9]:
#now I can create a gdf and set an active geom col and change crs to ft.
stops2 = gpd.GeoDataFrame(stops2).set_geometry('pt_geom').set_crs('EPSG:2229')

In [10]:
#function confirms that stops2 is a gdf, also used `stops2.geometry.name` and `stops2.crs` to confirm active geom col and crs was set as intended.
df_check(stops2)

#plotting reveals the stops2 is nationwide. may need to clip this to CA only or something.
stops2.geometry.name


'shape of df:(84688, 17)'

"type of :<class 'geopandas.geodataframe.GeoDataFrame'>"

,feed_key,stop_id,stop_key,stop_name,route_type_0,route_type_1,route_type_2,route_type_3,route_type_4,route_type_5,route_type_6,route_type_7,route_type_11,route_type_12,missing_route_type,geometry,pt_geom
0,6adf6cd9b6d24ab4ee8ee220e3697a73,15193,d4eb0920e7e256606df449c31b3c3e6a,Vanowen / Encino,NaN,NaN,NaN,69.0,NaN,NaN,None,None,None,None,None,"b'\x01\x01\x00\x00\x00?\xe5\x98,\xee\xa0]\xc0\...",POINT (-118.515 34.194)
1,6adf6cd9b6d24ab4ee8ee220e3697a73,14025,038cca58ef5f071ff5c94b8213989f87,Vermont / 110th,NaN,NaN,NaN,107.0,NaN,NaN,None,None,None,None,None,b'\x01\x01\x00\x00\x00\x10\x1em\x1c\xb1\x92]\x...,POINT (-118.292 33.936)
2,6adf6cd9b6d24ab4ee8ee220e3697a73,15638,06b1447efcc028791c8409d65fa3b3ee,3rd / Hobart,NaN,NaN,NaN,143.0,NaN,NaN,None,None,None,None,None,b'\x01\x01\x00\x00\x00\xd7d\x8dz\x88\x93]\xc03...,POINT (-118.305 34.069)
3,6adf6cd9b6d24ab4ee8ee220e3697a73,10244,87f19e30889f90d25e6dee49f04c4985,Vernon / Hooper,NaN,NaN,NaN,97.0,NaN,NaN,None,None,None,None,None,b'\x01\x01\x00\x00\x00z\xc2\x12\x0f(\x90]\xc0\...,POINT (-118.252 34.004)
4,6adf6cd9b6d24ab4ee8ee220e3697a73,20206,eda9e3eb339b7f510babcd4ee0999f85,Broadway / Pacific,NaN,NaN,NaN,108.0,NaN,NaN,None,None,None,None,None,b'\x01\x01\x00\x00\x001du\xab\xe7\x90]\xc0\xf4...,POINT (-118.264 34.147)


'pt_geom'

In [ ]:
df_check(stop_times)

In [12]:
#join stops2 and stop times, resulting df is for every stop, we get the stop time and its location 
stop2times = stops2.merge(stop_times, how='inner', on=['feed_key', 'stop_id'])

In [ ]:
#check to make sure merge works and that active geom col is pt_geom
df_check(stop2times)


## Calculate the straight line distance between the first and last stop for each trip. Call this column `trip_distance`



In [13]:
#testing pivot table for max stop sequence (aka last stop)

#for every trip, there are multiple stops. for every stop, there is a stop sequence 
#need coordinates for the stop!
pivot_max = stop2times.pivot_table(
    index='trip_id',
    values='stop_sequence',
    aggfunc={
        'stop_sequence':'max'}
).reset_index()

pivot_max

,trip_id,stop_sequence
0,002bqqucv,10
1,00339a54-1d52-408a-a0a5-db494ca7bef2,92
2,0049829c-505d-42db-965b-9ebd61bf1280,32
3,008-SCS-BLUE-01,24
4,008-SCS-BLUE-01a,24
...,...,...
100871,zaudfdl91,9
100872,zrgcyv1y8,10
100873,zrzuca41b,10
100874,zwn6ccb5f,10


In [29]:
#test to merge in pt_geom column from stop2times, based on stop sequence and trip id
pivot_max2 = pivot_max.merge(
    stop2times[['trip_id','pt_geom','stop_sequence']], 
    on=['trip_id','stop_sequence'],
    how='left'
)

In [30]:
pivot_max2 = pivot_max2.rename(columns={'stop_sequence':'last_stop', 'pt_geom':'last_stop_geom'})

In [31]:
#checking pivot merge to ensure column was renamed and have point geometry col
#note the resulting df is NOT A GDF
df_check(pivot_max2)


'shape of df:(100978, 3)'

"type of :<class 'pandas.core.frame.DataFrame'>"

,trip_id,last_stop,last_stop_geom
0,002bqqucv,10,POINT (-122.399 37.632)
1,00339a54-1d52-408a-a0a5-db494ca7bef2,92,POINT (-121.764 38.660)
2,0049829c-505d-42db-965b-9ebd61bf1280,32,POINT (-120.430 34.946)
3,008-SCS-BLUE-01,24,POINT (-122.444 37.664)
4,008-SCS-BLUE-01a,24,POINT (-122.444 37.664)


In [17]:
#using same pivot table method to get min stop value (aka first stop)
pivot_min = stop2times.pivot_table(
    index=['trip_id'],
    values=['stop_sequence'],
    aggfunc={
        'stop_sequence':'min'}
).reset_index()

pivot_min

,trip_id,stop_sequence
0,002bqqucv,1
1,00339a54-1d52-408a-a0a5-db494ca7bef2,0
2,0049829c-505d-42db-965b-9ebd61bf1280,1
3,008-SCS-BLUE-01,0
4,008-SCS-BLUE-01a,0
...,...,...
100871,zaudfdl91,1
100872,zrgcyv1y8,1
100873,zrzuca41b,1
100874,zwn6ccb5f,1


In [32]:
pivot_min2 = pivot_min.merge(
    stop2times[['trip_id', 'pt_geom', 'stop_sequence']],
    on=['trip_id','stop_sequence'],
    how='left').rename(columns={'stop_sequence':'first_stop', 'pt_geom': 'first_stop_geom'})

In [33]:
pivot_min2

,trip_id,first_stop,first_stop_geom
0,002bqqucv,1,POINT (-122.399 37.632)
1,00339a54-1d52-408a-a0a5-db494ca7bef2,0,POINT (-121.764 38.660)
2,0049829c-505d-42db-965b-9ebd61bf1280,1,POINT (-120.430 34.946)
3,008-SCS-BLUE-01,0,POINT (-122.444 37.665)
4,008-SCS-BLUE-01a,0,POINT (-122.444 37.665)
...,...,...,...
107337,zaudfdl91,1,POINT (-122.395 37.623)
107338,zrgcyv1y8,1,POINT (-122.399 37.632)
107339,zrzuca41b,1,POINT (-122.399 37.632)
107340,zwn6ccb5f,1,POINT (-122.399 37.632)


In [34]:
#steps to creat gdf
#stops2 = gpd.GeoDataFrame(stops2).set_geometry('pt_geom').set_crs('EPSG:2229')

def makegdf(df, geom):
    gdf = gpd.GeoDataFrame(df).set_geometry(geom).set_crs('EPSG:2229')
    
    return gdf

In [35]:
gdfmax = makegdf(pivot_max2, 'last_stop_geom')

gdfmin = makegdf(pivot_min2, 'first_stop_geom')

In [36]:
#check to ensure active geom col is set as intended, and crs is equivilent 
display(gdfmax.geometry.name)
display(gdfmin.geometry.name)
display(gdfmax.crs == gdfmin.crs)

'last_stop_geom'

'first_stop_geom'

True

In [37]:
df_check(gdfmax)
df_check(gdfmin)

'shape of df:(100978, 3)'

"type of :<class 'geopandas.geodataframe.GeoDataFrame'>"

,trip_id,last_stop,last_stop_geom
0,002bqqucv,10,POINT (-122.399 37.632)
1,00339a54-1d52-408a-a0a5-db494ca7bef2,92,POINT (-121.764 38.660)
2,0049829c-505d-42db-965b-9ebd61bf1280,32,POINT (-120.430 34.946)
3,008-SCS-BLUE-01,24,POINT (-122.444 37.664)
4,008-SCS-BLUE-01a,24,POINT (-122.444 37.664)


'shape of df:(107342, 3)'

"type of :<class 'geopandas.geodataframe.GeoDataFrame'>"

,trip_id,first_stop,first_stop_geom
0,002bqqucv,1,POINT (-122.399 37.632)
1,00339a54-1d52-408a-a0a5-db494ca7bef2,0,POINT (-121.764 38.660)
2,0049829c-505d-42db-965b-9ebd61bf1280,1,POINT (-120.430 34.946)
3,008-SCS-BLUE-01,0,POINT (-122.444 37.665)
4,008-SCS-BLUE-01a,0,POINT (-122.444 37.665)


In [38]:
#test to find the distance between pivot_max2 and pivot_min 2

distance = gdfmin.distance(gdfmax)

#value counts show we have values other than zero!
distance


/tmp/ipykernel_253/3783692448.py:3: UserWarning: The indices of the two GeoSeries are different.
  distance = gdfmin.distance(gdfmax)


0         0.000000
1         0.000000
2         0.000000
3         0.000492
4         0.000492
            ...   
107337         NaN
107338         NaN
107339         NaN
107340         NaN
107341         NaN
Length: 107342, dtype: float64

In [39]:
#test merge gdf pivots

testpv = gdfmin.merge(
    gdfmax,
    on='trip_id',
    how='left'
).assign(trip_distance = distance)

In [43]:
#checked `testpv` gdf, confirm that `trip_distance1 column is filled out, CRS got set to EPGS 229,
#also confirm by doing `value_counts` that we have unique distance values

df_check(testpv)
display(testpv.crs)
display(testpv.trip_distance.value_counts())

'shape of df:(107588, 6)'

"type of :<class 'geopandas.geodataframe.GeoDataFrame'>"

,trip_id,first_stop,first_stop_geom,last_stop,last_stop_geom,trip_distance
0,002bqqucv,1,POINT (-122.399 37.632),10,POINT (-122.399 37.632),0.000000
1,00339a54-1d52-408a-a0a5-db494ca7bef2,0,POINT (-121.764 38.660),92,POINT (-121.764 38.660),0.000000
2,0049829c-505d-42db-965b-9ebd61bf1280,1,POINT (-120.430 34.946),32,POINT (-120.430 34.946),0.000000
3,008-SCS-BLUE-01,0,POINT (-122.444 37.665),24,POINT (-122.444 37.664),0.000492
4,008-SCS-BLUE-01a,0,POINT (-122.444 37.665),24,POINT (-122.444 37.664),0.000492


<Projected CRS: EPSG:2229>
Name: NAD83 / California zone 5 (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - California - counties Kern; Los Angeles; San Bernardino; San Luis Obispo; Santa Barbara; Ventura.
- bounds: (-121.42, 32.76, -114.12, 35.81)
Coordinate Operation:
- name: SPCS83 California zone 5 (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

0.000000    3952
4.927793     324
0.139901     250
0.281782     234
0.222887     230
            ... 
1.623249       1
1.552449       1
0.111877       1
0.425197       1
5.763293       1
Name: trip_distance, Length: 32639, dtype: int64

## Calculate the distance between each stop to the nearest interstate. For each trip, keep the value for the stop that's the closest to the interstate. Call this column `shortest_distance_hwy`.

## Metholdolgy

someting about calculating distance from stop to 

test to see if it is possible to use .distance() from stops2 point geom and interstate multi-line geom?


In [ ]:
#distance_series = stops_geom.distance(interstates_geom)
testdist = 

In [85]:
#dissolve the highways by interstate. then buffer by .25 mi, then try to overlay stop points on interstate map, 

highway_d = highways.dissolve(by=['RouteType']).reset_index().to_crs('EPSG:2229')

In [86]:
df_check(highway_d)

'shape of df:(3, 6)'

"type of :<class 'geopandas.geodataframe.GeoDataFrame'>"

,RouteType,geometry,Route,County,District,Direction
0,Interstate,"MULTILINESTRING ((5405204.172 3674443.942, 540...",5,COL,3,NB
1,State,"MULTILINESTRING ((6518147.781 1746024.206, 651...",1,LA,7,NB
2,US,"MULTILINESTRING ((6459942.586 3062082.599, 645...",6,INY,9,NB


In [87]:
#reducing down the `highway_d` df to just interstate
#from highway_d, identify `routetype` column from `highway_d`, only take the value `interstate` from `routetype` column.
interstate = highway_d[highway_d['RouteType'] == 'Interstate']

In [90]:
df_check(interstate)
interstate.geometry.name

'shape of df:(1, 6)'

"type of :<class 'geopandas.geodataframe.GeoDataFrame'>"

,RouteType,geometry,Route,County,District,Direction
0,Interstate,"MULTILINESTRING ((5405204.172 3674443.942, 540...",5,COL,3,NB


'geometry'

In [89]:
# double checking both gdfs 
df_check(stops2)
df_check(interstate)

'shape of df:(84688, 17)'

"type of :<class 'geopandas.geodataframe.GeoDataFrame'>"

,feed_key,stop_id,stop_key,stop_name,route_type_0,route_type_1,route_type_2,route_type_3,route_type_4,route_type_5,route_type_6,route_type_7,route_type_11,route_type_12,missing_route_type,geometry,pt_geom
0,6adf6cd9b6d24ab4ee8ee220e3697a73,15193,d4eb0920e7e256606df449c31b3c3e6a,Vanowen / Encino,NaN,NaN,NaN,69.0,NaN,NaN,None,None,None,None,None,"b'\x01\x01\x00\x00\x00?\xe5\x98,\xee\xa0]\xc0\...",POINT (-118.515 34.194)
1,6adf6cd9b6d24ab4ee8ee220e3697a73,14025,038cca58ef5f071ff5c94b8213989f87,Vermont / 110th,NaN,NaN,NaN,107.0,NaN,NaN,None,None,None,None,None,b'\x01\x01\x00\x00\x00\x10\x1em\x1c\xb1\x92]\x...,POINT (-118.292 33.936)
2,6adf6cd9b6d24ab4ee8ee220e3697a73,15638,06b1447efcc028791c8409d65fa3b3ee,3rd / Hobart,NaN,NaN,NaN,143.0,NaN,NaN,None,None,None,None,None,b'\x01\x01\x00\x00\x00\xd7d\x8dz\x88\x93]\xc03...,POINT (-118.305 34.069)
3,6adf6cd9b6d24ab4ee8ee220e3697a73,10244,87f19e30889f90d25e6dee49f04c4985,Vernon / Hooper,NaN,NaN,NaN,97.0,NaN,NaN,None,None,None,None,None,b'\x01\x01\x00\x00\x00z\xc2\x12\x0f(\x90]\xc0\...,POINT (-118.252 34.004)
4,6adf6cd9b6d24ab4ee8ee220e3697a73,20206,eda9e3eb339b7f510babcd4ee0999f85,Broadway / Pacific,NaN,NaN,NaN,108.0,NaN,NaN,None,None,None,None,None,b'\x01\x01\x00\x00\x001du\xab\xe7\x90]\xc0\xf4...,POINT (-118.264 34.147)


'shape of df:(1, 6)'

"type of :<class 'geopandas.geodataframe.GeoDataFrame'>"

,RouteType,geometry,Route,County,District,Direction
0,Interstate,"MULTILINESTRING ((5405204.172 3674443.942, 540...",5,COL,3,NB


In [104]:
#test to see if i can calculate the distance from all rows in `stops2` to one geom `interstate`

distcheck = stops2.distance(interstate)


/tmp/ipykernel_253/3617348471.py:1: UserWarning: The indices of the two GeoSeries are different.
  distcheck = stops2.distance(interstate)


In [105]:
distcheck.value_counts()

6.153002e+06    1
dtype: int64

# hall-o-shame

In [ ]:
#df_check(highways)
#df_check(stop2times)
#stop2times.geometry.name

In [52]:
#interstate = interstate.geometry.buffer(1320)

In [ ]:
#clip = highways.to_crs('EPSG:2229').clip(districts)

#df_check(clip)
#clip.plot()

In [67]:
# not sure what to join
#sjoin = gpd.sjoin(
#    stops2,
 #   districts,
  #  how='inner',
   # predicate='intersects')
    #)

In [ ]:
#sjoin.plot()

In [ ]:
#merge pivot_max back into stop2times table to get the location of last stop
#stop2times_test = stop2times.merge(pivot_max, on=['trip_id','stop_id'],how='left')

In [ ]:
#df_check(stop2times_test)

In [ ]:
#merge pivot_min back into stop2times to get location of first stop.
#stop2times_test_2 = stop2times_test.merge(pivot_min, on=['trip_id','stop_id'], how='left')

In [ ]:
#df_check(stop2times_test_2)

In [ ]:
#test to calculate distance between first and last stop



In [ ]:
#cleaned up a couple of columns, dissolved by routes, reset index and set crs to feet
#highways_d = highways[['Route', 'geometry', 'RouteType']].dissolve(by='Route').reset_index().to_crs('EPSG:2229')

In [ ]:
#can you sjoin highways and stops to get stops in ca?

#sjoin = gpd.sjoin(highways.to_crs('EPSG:2229'), stops2, how='right')


#sjoin.plot()

In [ ]:
#df_check(highways_d)
#highways_d.plot()

In [ ]:
#test = stop2times({'trip_id': group.groups.keys(), 
#                   'first_stop': first_stop, 
#                  'last_stop': last_stop}
#                 )
#test.head

In [ ]:
#test to find the first stop of a trip usign .iloc[0]
#group = stop_times.groupby('trip_id')

In [ ]:
#first_stop = []
#last_stop = []

In [ ]:
# FOR LOOPS!!!
#for `every trip_id` group in group df, do this operation
#for trip_id, group in group:
#    f_stop = group.iloc[0]['stop_id']
#    l_stop = group.iloc[-1]['stop_id']

In [ ]:
#first_stop.append(f_stop)
#first_stop

In [ ]:
#last_stop.append(l_stop)
#last_stop

In [ ]:
#group.groups.keys()

In [ ]:

#add new col - first_stop. use assign with trip ID, and stop_sequence.iloc[0]
#add new col - last _stop. use assign with trip ID, and stop_sequence.iloc[-1]
#all new col - distance between first and last stop with distance first stop, last stop



In [ ]:
#overlay highways with stops2?
#RETURNS NOTHING!
#test = gpd.overlay(stops2, highways.to_crs('EPSG:2229'), how='intersection', keep_geom_type=True)

In [ ]:
#try to overlay stop2times on highways_d (points on line?)
#RETURNS NOTHING!
#test = gpd.overlay(highways.to_crs('EPSG:2229'), stop2times, how ='intersection', keep_geom_type=True)

In [ ]:
#df_check(test)

In [ ]:
#Calculate the distance between each stop to the nearest interstate. 
#For each trip, keep the value for the stop that's the closest to the interstate. Call this column shortest_distance_hwy.

In [ ]:
#can i dissolve by trip_ID, then get length?

#NOPE DIDNT WORK AS EXPECTED

#trip_d = stop2times.dissolve(by='trip_id').reset_index()

In [ ]:
#df_check(trip_d)
#trip_d.plot()